# Flux Partitioning with uncertainty prediction

This notebook aims at providing an uncertainty estimation for the flux partitioning problem, using Bayesian Neural Networks. This notebook only loads parameters from a previous run of HMC to plot some visulizations.
Thanks to Mohamed Aziz Bhouri for his help with Bayesian modeling and the HMC fine-tuning, and to Weiwei Zhan for her flux partitioning NN model and her data.

## Initialization and pre-training

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tqdm import tqdm

from HMC import HMC
from flux_preprocessing import load_dataset
from flux_utils import get_layer_model, fluxes_SIF_predict_noSIF, count_out_distribution
from flux_viz import quad_viz

%matplotlib inline
plt.rcParams["legend.frameon"] = False
plt.rcParams['savefig.dpi'] = 310
plt.rcParams['font.size'] = 13
data_dir = Path("../data")

tf.random.set_seed(0)

hidden_dim = 32

In [ ]:
train, test, EV1_train1, EV2_train1, NEE_train1, label_train, EV1_test1, EV2_test1, NEE_test1, label_test, NEE_max_abs = load_dataset(
    'NNinput_SCOPE_US_Ha1_1314.csv')

In [ ]:
def NN_noSIF(n_neuron, activation1, activation2):
    # GPP
    APAR_input = Input(shape=(1,), dtype='float32', name='APAR_input')
    EV_input1 = Input(shape=(EV1_train1.shape[1],), dtype='float32', name='EV_input1')
    x = Dense(n_neuron, activation=activation1, name='hidden1_1')(EV_input1)
    # x = Dense(n_neuron, activation=activation2, name='hidden1_2')(x)
    ln_GPP = Dense(1, activation=None, name='ln_GPP')(x)
    GPP_1 = Lambda(lambda x: K.exp(x), name='GPP_1')(ln_GPP)
    GPP = keras.layers.Multiply(name='GPP')([GPP_1, APAR_input])

    # Reco
    EV_input2 = Input(shape=(EV2_train1.shape[1],), dtype='float32', name='EV_input2')
    x = Dense(n_neuron, activation=activation1, name='hidden2_1')(EV_input2)
    x = Dense(n_neuron, activation=activation2, name='hidden2_2')(x)
    ln_Reco = Dense(1, activation=None, name='ln_Reco')(x)
    Reco = Lambda(lambda x: K.exp(x), name='Reco')(ln_Reco)

    NEE = keras.layers.Subtract(name='NEE')([Reco, GPP])

    model_NEE = Model(inputs=[APAR_input, EV_input1, EV_input2], outputs=[NEE])
    model_NEE.compile(
        optimizer=keras.optimizers.Adam(5e-3),
        loss=keras.losses.MeanSquaredError()
    )

    return model_NEE

In [ ]:
parameters_full = np.load("../etc/diagnostic/flux_nn/full_parameters.npy").astype(np.float32)
log_gamma_full = np.load("../etc/diagnostic/flux_nn/log_gamma.npy").astype(np.float32)
log_lambda_full = np.load("../etc/diagnostic/flux_nn/log_lambda.npy").astype(np.float32)
log_likelihood_full = np.load("../etc/diagnostic/flux_nn/log_likelihood.npy").astype(np.float32)

parameters, parameters2 = parameters_full[:,:,0], parameters_full[:,:,1]
log_gamma, log_gamma2 = log_gamma_full[:,0], log_gamma_full[:,1]
log_lambda, log_lambda2 = log_lambda_full[:,0], log_lambda_full[:,1]
log_likelihood, log_likelihood2 = log_likelihood_full[:,0], log_likelihood_full[:,1]

Here is a brief overview of the model that we are using. Depending on the specifications provided in the above function, the number  of trainable parameters is in the $10^3:10^4$ range.

In [ ]:
model1 = NN_noSIF(32,"relu","relu")
model1.summary()

## HMC run

Let's check on the MAP estimate, and compare the results with the training set.


In [ ]:
# Computing MAP estimate
n_obs_train = EV1_train1.shape[0]
n_obs_test = EV1_test1.shape[0]
hmc = HMC(model1, L=400, epsilon_min=5e-4, epsilon_max=5e-4, batch_size=n_obs_train, n_obs=n_obs_train)
idx_MAP = np.argmin(log_likelihood)
MAP = tf.constant(parameters[idx_MAP, :], dtype=tf.float32)
hmc.set_model_params(MAP)
NEE_train_MAP, GPP_train_MAP, Reco_train_MAP = fluxes_SIF_predict_noSIF(hmc.model, label_train, EV1_train1, EV2_train1,
                                                                        NEE_max_abs)
NEE_test_MAP, GPP_test_MAP, Reco_test_MAP = fluxes_SIF_predict_noSIF(hmc.model, label_test, EV1_test1, EV2_test1,
                                                                     NEE_max_abs)

train_pred = hmc.model({'APAR_input': label_train, 'EV_input1': EV1_train1, 'EV_input2': EV2_train1})
train_loss = hmc.model.loss(train_pred, NEE_train1)
plt.figure(figsize=(15,10))
plt.scatter(train_pred, NEE_train1)
plt.show()

test_pred = hmc.model({'APAR_input': label_test, 'EV_input1': EV1_test1, 'EV_input2': EV2_test1})
test_loss = hmc.model.loss(test_pred, NEE_test1)
print("MAP training loss = {train_loss:.3e} and testing loss = {test_loss:.3e}".format(train_loss = train_loss, test_loss=test_loss))

## HMC sampling


In [ ]:
# preparing sampling
precision = np.exp(log_gamma)
n_samples = 500
n_iter = parameters_full.shape[0]
NEE_train_traj, GPP_train_traj, Reco_train_traj = np.zeros((n_obs_train, n_samples)), np.zeros(
    (n_obs_train, n_samples)), np.zeros(
    (n_obs_train, n_samples))
NEE_test_traj, GPP_test_traj, Reco_test_traj = np.zeros((n_obs_test, n_samples)), np.zeros(
    (n_obs_test, n_samples)), np.zeros(
    (n_obs_test, n_samples))
sigma_NEE = np.std(train["NEE_canopy"])
sigma_GPP = np.std(train["GPP_canopy"])
sigma_Reco = np.std(train["Reco_canopy"])

# sampling
model_NEE = get_layer_model(hmc.model, "NEE")
model_GPP = get_layer_model(hmc.model, "GPP")
model_Reco = get_layer_model(hmc.model, "Reco")
for k in tqdm(range(n_samples)):
    idx_1 = np.random.randint(0, n_iter)
    idx_2 = np.random.randint(0, n_iter)
    w_sample = parameters[-idx_1, :]
    precision_here = precision[-idx_1]
    hmc.set_model_params(w_sample)

    NEE_train_traj[:, k] = NEE_max_abs * tf.squeeze(model_NEE({'APAR_input': label_train,
                                                               'EV_input1': EV1_train1,
                                                               'EV_input2': EV2_train1})) + sigma_NEE * np.random.normal() / np.sqrt(
        precision_here)
    GPP_train_traj[:, k] = NEE_max_abs * tf.squeeze(model_GPP({'APAR_input': label_train,
                                                               'EV_input1': EV1_train1,
                                                               'EV_input2': EV2_train1})) + sigma_GPP * np.random.normal() / np.sqrt(
        precision_here)
    Reco_train_traj[:, k] = NEE_max_abs * tf.squeeze(model_Reco({'APAR_input': label_train,
                                                                 'EV_input1': EV1_train1,
                                                                 'EV_input2': EV2_train1})) + sigma_Reco * np.random.normal() / np.sqrt(
        precision_here)

    NEE_test_traj[:, k] = NEE_max_abs * tf.squeeze(model_NEE({'APAR_input': label_test,
                                                              'EV_input1': EV1_test1,
                                                              'EV_input2': EV2_test1})) + sigma_NEE * np.random.normal() / np.sqrt(
        precision_here)
    GPP_test_traj[:, k] = NEE_max_abs * tf.squeeze(model_GPP({'APAR_input': label_test,
                                                              'EV_input1': EV1_test1,
                                                              'EV_input2': EV2_test1})) + sigma_GPP * np.random.normal() / np.sqrt(
        precision_here)
    Reco_test_traj[:, k] = NEE_max_abs * tf.squeeze(model_Reco({'APAR_input': label_test,
                                                                'EV_input1': EV1_test1,
                                                                'EV_input2': EV2_test1})) + sigma_Reco * np.random.normal() / np.sqrt(
        precision_here)

mu_NEE_pred_train = np.mean(NEE_train_traj, axis=1)
mu_GPP_pred_train = np.mean(GPP_train_traj, axis=1)
mu_Reco_pred_train = np.mean(Reco_train_traj, axis=1)

sigma_NEE_pred_train = np.std(NEE_train_traj, axis=1)
sigma_GPP_pred_train = np.std(GPP_train_traj, axis=1)
sigma_Reco_pred_train = np.std(Reco_train_traj, axis=1)

mu_NEE_pred_test = np.mean(NEE_test_traj, axis=1)
mu_GPP_pred_test = np.mean(GPP_test_traj, axis=1)
mu_Reco_pred_test = np.mean(Reco_test_traj, axis=1)

sigma_NEE_pred_test = np.std(NEE_test_traj, axis=1)
sigma_GPP_pred_test = np.std(GPP_test_traj, axis=1)
sigma_Reco_pred_test = np.std(Reco_test_traj, axis=1)

## Diagnostics

I run some diagnostics here (mostly auto-correlation). A more complete set of diagnostics is conducted in the corresponding R notebook.
The first step is to conduct a second HMC walk, with a different initialization, to check on the convergence.

Let's then check with 16 random parameters that the auto-correlation plot doesn't show any symptomatic behavior.

In [ ]:
# plot auto-correlation

fig, axs = plt.subplots(4, 4, figsize=(12, 10))
axs = axs.flatten()
max_lag = int(n_iter/4)
for k in range(len(axs)):
    i = np.random.randint(0, parameters.shape[1])
    auto_cor = []
    mean = np.mean(parameters[:, i])
    var = np.var(parameters[:, i])
    for lag in range(1, max_lag):
        param = parameters[lag:, i]
        param_lagged = parameters[:-lag, i]
        auto_cor.append(np.mean((param - mean) * (param_lagged - mean)) / var)
    axs[k].plot(np.arange(1, max_lag), auto_cor)
plt.tight_layout()
plt.show()

We conduct the same operation with $\log(\gamma)$ and $\log(\lambda)$. These are more prone to show high-correlation, especially $\log(\lambda)$, as explained earlier. If that's the case, I would suggest to slightly increase `epsilon` as long as the acceptance rate doesn't drop, and then increase `L`.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs = axs.flatten()

mean = np.mean(log_gamma)
var = np.var(log_gamma)
auto_cor = []
for lag in range(1, max_lag):
    param = log_gamma[lag:]
    param_lagged = log_gamma[:-lag]
    auto_cor.append(np.mean((param - mean) * (param_lagged - mean)) / var)
axs[0].plot(np.arange(1, max_lag), auto_cor)

mean = np.mean(log_lambda)
var = np.var(log_lambda)
auto_cor = []
for lag in range(1, max_lag):
    param = log_lambda[lag:]
    param_lagged = log_lambda[:-lag]
    auto_cor.append(np.mean((param - mean) * (param_lagged - mean)) / var)
axs[1].plot(np.arange(1, max_lag), auto_cor)
plt.show()

We can also have a look at the parameter spread and correlation. It should be close to a centered gaussian in most cases.

In [ ]:
# plot parameters spread
fig, axs = plt.subplots(5, 5, figsize=(20, 15))
for i in range(axs.shape[0]):
    axs[i, i].hist(parameters[:, i], bins=30, density=True)
    for j in range(i):
        axs[i, j].scatter(parameters[:, j], parameters[:, i], s=1, marker="+", c=np.arange(parameters.shape[0]))
    for j in range(i + 1, axs.shape[0]):
        axs[i, j].hist2d(parameters[:, j], parameters[:, i], bins=30, cmap="Blues")
plt.tight_layout()
plt.show()

## Predictions

Let's now have a look at the computed predictions and uncertainties.

In [ ]:
# Logging predictions in their respective dataframes
test['NEE_MAP'] = NEE_test_MAP
test['Reco_MAP'] = Reco_test_MAP
test['GPP_MAP'] = GPP_test_MAP
train['NEE_MAP'] = NEE_train_MAP
train['Reco_MAP'] = Reco_train_MAP
train['GPP_MAP'] = GPP_train_MAP

test['NEE_mean'] = mu_NEE_pred_test
test['Reco_mean'] = mu_Reco_pred_test
test['GPP_mean'] = mu_GPP_pred_test
train['NEE_mean'] = mu_NEE_pred_train
train['Reco_mean'] = mu_Reco_pred_train
train['GPP_mean'] = mu_GPP_pred_train

test['NEE_sigma'] = sigma_NEE_pred_test
test['Reco_sigma'] = sigma_Reco_pred_test
test['GPP_sigma'] = sigma_GPP_pred_test
train['NEE_sigma'] = sigma_NEE_pred_train
train['Reco_sigma'] = sigma_Reco_pred_train
train['GPP_sigma'] = sigma_GPP_pred_train

train_day = train.loc[train.APAR_label == 1,]
train_night = train.loc[train.APAR_label == 0,]
test_day = test.loc[test.APAR_label == 1,]
test_night = test.loc[test.APAR_label == 0,]

Usually the respiration gets the worst performance. I tried different model architectures and this seems to be the "best" one (even though it's far from optimal). I think 2 effects are conjugated here:

- The respiration has a lower amplitude than the GPP, thus the model primary focuses on the GPP as it's responsible for a larger part of the loss;
- The Laplace prior for the model parameters is enforcing too much sparsity, which harms the performances;

On the left, the entire training and testing sets are shown (it's not the best as they are quite large, I'll work on that). On the right the MAP estimates are plotted against the observed values.

The MAP estimate seems to capture the global trend, but is quite inaccurate. The uncertainty is so high that the predictions are almost unusable.

In [ ]:
fig, ax = quad_viz(train, test, "Reco", colors="Tsoil", unit="(umol m-2 s-1)", filename="../etc/images/reco_nn.png")
print("Ratio of training observations outside the 2σ band: {r:.2e}".format(r=count_out_distribution(train_day, "Reco") / train_day.shape[0]))
print("Ratio of testing observations outside the 2σ band: {r:.2e}".format(r=count_out_distribution(test_day, "Reco") / train_day.shape[0]))

Results are much better for the GPP part. Again, the data sets are too large to have a nice time-series plot. But here, the MAP estimates are much more accurate, with a high $R^2$, and when zooming on the test set, the uncertainty seem to be completely reasonable.

In [ ]:
fig, ax = quad_viz(train, test, "GPP", filename="../etc/images/gpp_nn.png")
print("Ratio of training observations outside the 2σ band: {r:.2e}".format(r=count_out_distribution(train_day, "GPP") / train_day.shape[0]))
print("Ratio of testing observations outside the 2σ band: {r:.2e}".format(r=count_out_distribution(test_day, "GPP") / train_day.shape[0]))

For control, here is the NEE trajectory. The performances are good overall, despite the poor results for Reco, which seems to confirm that the modeling choices are giving too much weight to GPP.

In [ ]:
fig, ax = quad_viz(train, test, "NEE", filename="../etc/images/nee_nn.png")
print("Ratio of training observations outside the 2σ band: {r:.2e}".format(r=count_out_distribution(train_day, "NEE") / train_day.shape[0]))
print("Ratio of testing observations outside the 2σ band: {r:.2e}".format(r=count_out_distribution(test_day, "NEE") / train_day.shape[0]))

## $L_1$ regularization

At first, I thought that the Bayesian modeling was enforcing too much sparsity on the parameters. Thus, I tried to train a few NN models with different $L_1$ regularization. It is not as conclusive as I expected, but it's not enough to reject the hypothesis either.

Without any regularization, the predictions are much closer than our previous MAP estimate.

In [ ]:
# GPP
APAR_input = Input(shape=(1,), dtype='float32', name='APAR_input')
EV_input1 = Input(shape=(EV1_train1.shape[1],), dtype='float32', name='EV_input1')
x = Dense(hidden_dim, activation="relu", name='hidden1_1')(EV_input1)
ln_GPP = Dense(1, activation=None, name='ln_GPP')(x)
GPP_1 = Lambda(lambda x: K.exp(x), name='GPP_1')(ln_GPP)
GPP = keras.layers.Multiply(name='GPP')([GPP_1, APAR_input])

# Reco
EV_input2 = Input(shape=(EV2_train1.shape[1],), dtype='float32', name='EV_input2')
x = Dense(hidden_dim, activation="relu", name='hidden2_1')(EV_input2)
x = Dense(hidden_dim, activation="relu", name='hidden2_2')(x)
ln_Reco = Dense(1, activation=None, name='ln_Reco')(x)
Reco = Lambda(lambda x: K.exp(x), name='Reco')(ln_Reco)

NEE = keras.layers.Subtract(name='NEE')([Reco, GPP])

model_NEE = Model(inputs=[APAR_input, EV_input1, EV_input2], outputs=[NEE])
model_NEE.compile(
    optimizer=keras.optimizers.Adam(2e-3),
    loss=keras.losses.MeanSquaredError()
)
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
model_NEE.fit({'APAR_input': label_train, 'EV_input1': EV1_train1, 'EV_input2': EV2_train1},
                             {'NEE': NEE_train1},
                             epochs=500,
                             batch_size=64,
                             callbacks=[earlyStopping],
                             validation_split=0.3,
                             verbose=0)
NEE_train_NN, GPP_train_NN, Reco_train_NN = fluxes_SIF_predict_noSIF(model_NEE, label_train, EV1_train1, EV2_train1, NEE_max_abs)
NEE_test_NN, GPP_test_NN, Reco_test_NN = fluxes_SIF_predict_noSIF(model_NEE, label_test, EV1_test1, EV2_test1, NEE_max_abs)
test['NEE_MAP'] = NEE_test_NN
test['Reco_MAP'] = Reco_test_NN
test['GPP_MAP'] = GPP_test_NN
train['NEE_MAP'] = NEE_train_NN
train['Reco_MAP'] = Reco_train_NN
train['GPP_MAP'] = GPP_train_NN
fig, ax = quad_viz(train, test, "Reco", colors="Tsoil", unit="(umol m-2 s-1)", bayesian=False)
None

With a weight of $10^{-4}$ on all the parameters, performance actually increases.

In [ ]:
# GPP
APAR_input = Input(shape=(1,), dtype='float32', name='APAR_input')
EV_input1 = Input(shape=(EV1_train1.shape[1],), dtype='float32', name='EV_input1')
x = Dense(hidden_dim, activation="relu", name='hidden1_1', kernel_regularizer=tf.keras.regularizers.l1(1e-4), bias_regularizer=tf.keras.regularizers.l1(1e-4))(EV_input1)
ln_GPP = Dense(1, activation=None, name='ln_GPP')(x)
GPP_1 = Lambda(lambda x: K.exp(x), name='GPP_1')(ln_GPP)
GPP = keras.layers.Multiply(name='GPP')([GPP_1, APAR_input])

# Reco
EV_input2 = Input(shape=(EV2_train1.shape[1],), dtype='float32', name='EV_input2')
x = Dense(hidden_dim, activation="relu", name='hidden2_1', kernel_regularizer=tf.keras.regularizers.l1(1e-4), bias_regularizer=tf.keras.regularizers.l1(1e-4))(EV_input2)
x = Dense(hidden_dim, activation="relu", name='hidden2_2', kernel_regularizer=tf.keras.regularizers.l1(1e-4), bias_regularizer=tf.keras.regularizers.l1(1e-4))(x)
ln_Reco = Dense(1, activation=None, name='ln_Reco')(x)
Reco = Lambda(lambda x: K.exp(x), name='Reco')(ln_Reco)

NEE = keras.layers.Subtract(name='NEE')([Reco, GPP])

model_NEE = Model(inputs=[APAR_input, EV_input1, EV_input2], outputs=[NEE])
model_NEE.compile(
    optimizer=keras.optimizers.Adam(2e-3),
    loss=keras.losses.MeanSquaredError()
)
model_NEE.fit({'APAR_input': label_train, 'EV_input1': EV1_train1, 'EV_input2': EV2_train1},
                             {'NEE': NEE_train1},
                             epochs=500,
                             batch_size=64,
                             callbacks=[earlyStopping],
                             validation_split=0.3,
                             verbose=0)
NEE_train_NN, GPP_train_NN, Reco_train_NN = fluxes_SIF_predict_noSIF(model_NEE, label_train, EV1_train1, EV2_train1, NEE_max_abs)
NEE_test_NN, GPP_test_NN, Reco_test_NN = fluxes_SIF_predict_noSIF(model_NEE, label_test, EV1_test1, EV2_test1, NEE_max_abs)
test['NEE_MAP'] = NEE_test_NN
test['Reco_MAP'] = Reco_test_NN
test['GPP_MAP'] = GPP_test_NN
train['NEE_MAP'] = NEE_train_NN
train['Reco_MAP'] = Reco_train_NN
train['GPP_MAP'] = GPP_train_NN
fig, ax = quad_viz(train, test, "Reco", colors="Tsoil", unit="(umol m-2 s-1)", bayesian=False)
None

However, with more regularization ($> 3\times 10^{-4}$) the predictions become almost constant with value 4.

In [ ]:
# GPP
APAR_input = Input(shape=(1,), dtype='float32', name='APAR_input')
EV_input1 = Input(shape=(EV1_train1.shape[1],), dtype='float32', name='EV_input1')
x = Dense(hidden_dim, activation="relu", name='hidden1_1', kernel_regularizer=tf.keras.regularizers.l1(3e-4), bias_regularizer=tf.keras.regularizers.l1(3e-4))(EV_input1)
ln_GPP = Dense(1, activation=None, name='ln_GPP')(x)
GPP_1 = Lambda(lambda x: K.exp(x), name='GPP_1')(ln_GPP)
GPP = keras.layers.Multiply(name='GPP')([GPP_1, APAR_input])

# Reco
EV_input2 = Input(shape=(EV2_train1.shape[1],), dtype='float32', name='EV_input2')
x = Dense(hidden_dim, activation="relu", name='hidden2_1', kernel_regularizer=tf.keras.regularizers.l1(3e-4), bias_regularizer=tf.keras.regularizers.l1(3e-4))(EV_input2)
x = Dense(hidden_dim, activation="relu", name='hidden2_2', kernel_regularizer=tf.keras.regularizers.l1(3e-4), bias_regularizer=tf.keras.regularizers.l1(3e-4))(x)
ln_Reco = Dense(1, activation=None, name='ln_Reco')(x)
Reco = Lambda(lambda x: K.exp(x), name='Reco')(ln_Reco)

NEE = keras.layers.Subtract(name='NEE')([Reco, GPP])

model_NEE = Model(inputs=[APAR_input, EV_input1, EV_input2], outputs=[NEE])
model_NEE.compile(
    optimizer=keras.optimizers.Adam(2e-3),
    loss=keras.losses.MeanSquaredError()
)
model_NEE.fit({'APAR_input': label_train, 'EV_input1': EV1_train1, 'EV_input2': EV2_train1},
                             {'NEE': NEE_train1},
                             epochs=500,
                             batch_size=64,
                             callbacks=[earlyStopping],
                             validation_split=0.3,
                             verbose=0)
NEE_train_NN, GPP_train_NN, Reco_train_NN = fluxes_SIF_predict_noSIF(model_NEE, label_train, EV1_train1, EV2_train1, NEE_max_abs)
NEE_test_NN, GPP_test_NN, Reco_test_NN = fluxes_SIF_predict_noSIF(model_NEE, label_test, EV1_test1, EV2_test1, NEE_max_abs)
test['NEE_MAP'] = NEE_test_NN
test['Reco_MAP'] = Reco_test_NN
test['GPP_MAP'] = GPP_test_NN
train['NEE_MAP'] = NEE_train_NN
train['Reco_MAP'] = Reco_train_NN
train['GPP_MAP'] = GPP_train_NN
fig, ax = quad_viz(train, test, "Reco", colors="Tsoil", unit="(umol m-2 s-1)", bayesian=False)
None

While it's a nice warning on the effect of over-regularization, predictions don't look anything like our previous MAP predictions. So we cannot conclude on the benefits of the sparsity brought by the Laplace prior.